## Landscape Approximation

### C-Shaped Plateau Function Description

The C-shaped plateau region is a benchmark problem designed to test the performance of optimization algorithms, particularly in regions where the fitness landscape has plateaus or low-gradient areas. The domain for this test is defined as $\mathcal{D} = [-3, 3]^2$, and the fitness function, denoted as $f_C(x)$, is constructed as the product of Gaussian-based functions.

#### Fitness Function Definition

The benchmark fitness function $f_C(x)$ is mathematically represented as:

$$f_C(x) = g^{(1, 0.5)}_{(0, 1.5)}(x) \cdot g^{(0.5, 1)}_{(1.5, 0)}(x) \cdot g^{(1, 0.5)}_{(0, -1.5)}(x)$$

where $g^{(a,b)}_{(c,d)}(x)$ represents a Gaussian function centered at the point $(c,d)$ with covariance parameters $a$ and $b$ determining the spread along the respective axes. The plateau formed by this function $f_C(x)$ creates a C-shaped valley, which comprises 6.2% of the entire domain. This specific shape and coverage make it a challenging landscape for optimization algorithms, as they must effectively explore and navigate the flat regions of the landscape.

In [ ]:
import numpy as np


def gaussian_benchmark(x, x0, r):
    """
    Calculate the Gaussian-based benchmark function.

    Parameters:
    x (numpy array): Input vector.
    x0 (numpy array): Center vector.
    r (numpy array): Vector of parameters determining the spread along each axis.

    Returns:
    float: Value of the benchmark function.
    """
    # Create diagonal matrix S with S_{i,i} = 1/r_i
    S = np.diag(1.0 / r)
    diff = x - x0
    exponent = -np.dot(np.dot(diff.T, S), diff)
    return 1 - np.exp(exponent)


def flat_transform(s):
    """
    Apply flattening transformation to function value.
    
    Parameters:
    s (float): Function value to transform.
    
    Returns:
    float: Transformed value max(2s-1, 0).
    """
    return max(2 * s - 1, 0)


def c_shaped_plateau(x):
    """
    Calculate the C-shaped plateau function f_C(x) with flattening.

    Parameters:
    x (numpy array): Input vector.

    Returns:
    float: Value of the C-shaped plateau function after flattening.
    """
    # Define the r parameters for each Gaussian
    r_params = [
        np.array([1.0, 0.5]),
        np.array([0.5, 1.0]),
        np.array([1.0, 0.5]),
    ]

    # Define the center vectors
    centers = [
        np.array([0.0, 1.5]),
        np.array([1.5, 0.0]),
        np.array([0.0, -1.5]),
    ]

    # Compute the product of all g functions
    f_C = 1.0
    for x0, r in zip(centers, r_params):
        f_C *= gaussian_benchmark(x, x0, r)

    # Apply the flattening transformation
    return flat_transform(f_C)

#### HMS
We prepare an HMS config and we run it with a budget of 10000 evaluations.

In [ ]:
from pyhms.core.problem import EvalCountingProblem, FunctionProblem
from pyhms import (
    EALevelConfig,
    DontStop,
    MetaepochLimit,
    SingularProblemEvalLimitReached,
    SEA,
    CMALevelConfig,
    TreeConfig,
    DemeTree,
)
from pyhms.sprout import get_NBC_sprout
from pyhms.demes.single_pop_eas.sea import SEA

N = 2
bounds = np.array([(-3, 3)] * N)

problem = FunctionProblem(c_shaped_plateau, maximize=False, bounds=bounds)
counting_problem = EvalCountingProblem(problem)
tree_config = [
    EALevelConfig(
        ea_class=SEA,
        generations=1,
        problem=counting_problem,
        pop_size=100,
        mutation_std=1.0,
        k_elites=5,
        election_group_size=30,
        lsc=DontStop(),
        p_mutation=0.8,
        p_crossover=0.3,
        mutation_std_step=0.1,
    ),
    CMALevelConfig(
        generations=20, problem=counting_problem, lsc=MetaepochLimit(10), sigma0=1.0
    ),
]

global_stop_condition = SingularProblemEvalLimitReached(10000)

sprout_condition = get_NBC_sprout()
config = TreeConfig(
    tree_config, global_stop_condition, sprout_condition, options={"random_seed": 1}
)
hms_tree = DemeTree(config)
hms_tree.run()

#### Landscape Approximation
We use a default `LandscapeApproximator` with `HillValleyMergeCondition`.

In [ ]:
from pyhms.cluster.landscape_approximator import LandscapeApproximator
from pyhms.cluster.merge_conditions import (
    HillValleyMergeCondition,
)
from pyhms.demes.single_pop_eas.sea import MWEA

problem = FunctionProblem(c_shaped_plateau, maximize=False, bounds=bounds)

mwea = MWEA.create(problem=problem, mutation_std=0.2, p_mutation=0.35)

la = LandscapeApproximator(
    hms_tree=hms_tree,
    merge_condition=HillValleyMergeCondition(
        FunctionProblem(c_shaped_plateau, maximize=False, bounds=bounds), k=10
    ),
    local_basin_epochs=10,
    mwea=mwea,
)
la.fit()
la.plot(filepath="kriging.eps")

#### Problem visualization

In [ ]:
hms_tree.plot_problem_surface()

#### Local Basin Populations

In [ ]:
import plotly.express as px

reduced_clusters = la.cluster_reducer.reduce_clusters(la.clusters)
local_basin_population = la.local_basin_agent_executor(reduced_clusters)

px.scatter(
    x=local_basin_population.genomes[:, 0],
    y=local_basin_population.genomes[:, 1],
)